In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings

from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 13

import sys
sys.path.insert(0, '../cmds')
from treasury_cmds import *

In [2]:
import pandas_datareader.data as web
import pandas_datareader as pdr

import yfinance as yf

In [3]:
QDATE = '2022-07-20'
TICK = 'TSLA'
sec = yf.Ticker(TICK)

In [4]:
START_DATE = '2022-07-15'
END_DATE = '2022-07-21'
TICKRF = 'SOFR'
sofr = pdr.DataReader(TICKRF,data_source='fred', start=START_DATE,end=END_DATE)

quotes = sofr.loc[QDATE]/100

In [5]:
tbill = yf.Ticker('^IRX')
quotes.loc['TBill (3mo)'] = tbill.history('1d')['Close'].iloc[-1]/100

px = sec.history('1d',start=START_DATE,end=END_DATE)['Close']
quotes.loc[f'{TICK}'] = px.loc[QDATE]

quotes = quotes.to_frame().T

# Options

In [6]:
def filter_stale_quotes(opt_chain):
    LDATE =opt_chain.sort_values('lastTradeDate')['lastTradeDate'].iloc[-1]
    mask = list()

    for idx in opt_chain.index:
        dt = opt_chain.loc[idx,'lastTradeDate']
        if (dt - LDATE).total_seconds()/3600 > -24:
            mask.append(idx)
    
    return mask

In [7]:
def clean_options(calls_raw,puts_raw):
    idx = filter_stale_quotes(calls_raw)
    calls = calls_raw.loc[idx,:]
    idx = filter_stale_quotes(puts_raw)
    puts = puts_raw.loc[idx,:]

    calls = calls[calls['volume'] > calls['volume'].quantile(.75)].set_index('contractSymbol')
    puts = puts[puts['volume'] > puts['volume'].quantile(.75)].set_index('contractSymbol')
    
    calls['lastTradeDate'] = calls['lastTradeDate'].dt.tz_localize(None)
    puts['lastTradeDate'] = puts['lastTradeDate'].dt.tz_localize(None)
    
    return calls, puts

In [8]:
EXPRYDATE = '2022-08-19'
opts = sec.option_chain(EXPRYDATE)
calls, puts = clean_options(opts.calls, opts.puts)

In [9]:
EXPRYDATE_T = '2022-12-16'
opts = sec.option_chain(EXPRYDATE)
callsT, putsT = clean_options(opts.calls, opts.puts)

In [10]:
#tau = (pd.to_datetime(EXPRYDATE) - pd.to_datetime(QDATE)).total_seconds()/(24*60*60)/365

# Export

In [11]:
outfile = '../data/options_chains.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    quotes.to_excel(writer, sheet_name= f'Market Quotes {QDATE}', index=True)
    calls.to_excel(writer, sheet_name= f'{TICK} CALLS {EXPRYDATE}', index=True)
    puts.to_excel(writer, sheet_name= f'{TICK} PUTS {EXPRYDATE}', index=True)
    callsT.to_excel(writer, sheet_name= f'{TICK} CALLS {EXPRYDATE_T}', index=True)
    putsT.to_excel(writer, sheet_name= f'{TICK} PUTS {EXPRYDATE_T}', index=True)